<a href="https://colab.research.google.com/github/aswanthoffl/medical_chatbot/blob/master/chatbot_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import json
import pickle
import numpy as np1
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer=WordNetLemmatizer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
with open(r'/content/drive/MyDrive/Colab Notebooks/Dataset/intents.json') as json_file:
    intents = json.load(json_file)

print(intents)


{'intents': [{'tag': 'greetings', 'patterns': ['hello', 'hey', 'hi', 'good day', 'greetings', "what's up?", 'how is it going'], 'responses': ['hello', 'hey!', 'what can i do for you?']}, {'tag': 'goodbye', 'patterns': ['cya', 'see you later', 'goodbye', 'have a good day', 'bye', 'cao', 'see ya'], 'responses': ['have a nice day', 'goodbye']}, {'tag': 'age', 'patterns': ['how old', 'how old are you?', 'what is your age', 'how old are you', 'age?'], 'responses': ['I get reborn after every compilation', 'hey!', 'my owners are averagely 20 years!']}, {'tag': 'name', 'patterns': ['what is your name', 'what should i call you', "what's your name?", 'who are you?', 'can you tell me your name'], 'responses': ['you can call me Medbot!', 'i am Medbot!', 'i am Medbot your medical assistant']}, {'tag': 'common cold symptoms', 'patterns': ['Runny or stuffy nose', 'Sore throat', 'Cough', 'Congestion', 'Slight body aches or a mild headache', 'Sneezing', 'Low-grade fever', 'Generally feeling unwell (mal

In [ ]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']


for intent in intents['intents']:
  for pattern in intent['patterns']:
    # Convert pattern to lowercase
    pattern = pattern.lower()
    word_list=nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])



for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list=nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


words =[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes=sorted(set(classes))
pickle.dump(words,open('/content/drive/MyDrive/Colab Notebooks/Dataset/words2.pkl','wb'))
pickle.dump(classes,open('/content/drive/MyDrive/Colab Notebooks/Dataset/classes2.pkl','wb'))


In [ ]:
training=[]
output_empty=[0]*len(classes)

for document in documents:
  bag=[]
  word_patterns=document[0]
  words = [lemmatizer.lemmatize(word) for word in words if word and word not in ignore_letters]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row=list(output_empty)
  output_row[classes.index(document[1])]=1
  training.append([bag,output_row])


In [ ]:
random.shuffle(training)

train_x = np.array([x[0] for x in training])
train_y = np.array([x[1] for x in training])

In [ ]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))


In [ ]:
 #sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
hist = model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save(r'/content/drive/MyDrive/Colab Notebooks/Dataset/chatbotmodel2.h5', hist)
print('Training Done')

Epoch 1/200
34/34 [==============================] - 1s 3ms/step - loss: 2.7446 - accuracy: 0.0714
Epoch 2/200
34/34 [==============================] - 0s 2ms/step - loss: 2.6461 - accuracy: 0.1250
Epoch 3/200
34/34 [==============================] - 0s 2ms/step - loss: 2.4950 - accuracy: 0.1607
Epoch 4/200
34/34 [==============================] - 0s 2ms/step - loss: 2.3450 - accuracy: 0.2262
Epoch 5/200
34/34 [==============================] - 0s 4ms/step - loss: 2.1520 - accuracy: 0.3155
Epoch 6/200
34/34 [==============================] - 0s 4ms/step - loss: 1.9318 - accuracy: 0.3631
Epoch 7/200
34/34 [==============================] - 0s 5ms/step - loss: 1.7256 - accuracy: 0.4345
Epoch 8/200
34/34 [==============================] - 0s 4ms/step - loss: 1.5628 - accuracy: 0.5238
Epoch 9/200
34/34 [==============================] - 0s 4ms/step - loss: 1.4006 - accuracy: 0.5536
Epoch 10/200
34/34 [==============================] - 0s 4ms/step - loss: 1.3603 - accuracy: 0.5060
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Training Done
